In [1]:
import torch
from torchvision import models, transforms
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import os
import cv2
import keras
import warnings
import time
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import random

## Load dog breed and dog classification

In [2]:
def loading_training_data(data_dir):
    data = []
    labels_list = []
    
    for label in labels:
        # path = os.path.join(data_dir, label)
        path = data_dir+"/"+label
        class_num = labels.index(label)
        for img in os.listdir(path):
            # print(img)
            # print(path)
            # img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
            img_arr = cv2.imread(os.path.join(path, img))
            
            # print(os.path.join(path,img))
            if img_arr is None:
                # print(f"Warning: Unable to load image {img}")
                print(os.path.join(path,img))
                continue  # Skip the image if it can't be loaded
            resized_arr = cv2.resize(img_arr, (img_size, img_size))
            data.append(resized_arr)
            labels_list.append(class_num)
            
    return np.array(data), np.array(labels_list)


In [37]:
def loading_training_data_subset(data_dir, subset_size=None):
    data = []
    labels_list = []
    
    for label in labels:
        # path = os.path.join(data_dir, label)  # Assuming label directories are in data_dir
        path = data_dir+"/"+label
        class_num = labels.index(label)
        
        # Get list of images in the directory
        all_images = os.listdir(path)
        
        # If subset_size is provided, select a random subset of images
        if subset_size is not None and len(all_images) > subset_size:
            all_images = random.sample(all_images, subset_size)
        
        for img in all_images:
            img_arr = cv2.imread(os.path.join(path, img))
            
            if img_arr is None:
                print(f"Warning: Unable to load image {img}")
                continue  # Skip the image if it can't be loaded
            
            resized_arr = cv2.resize(img_arr, (img_size, img_size))
            data.append(resized_arr)
            labels_list.append(class_num)
    
    return np.array(data), np.array(labels_list)

In [10]:
def transform_input_resnet(data):
    # Convert teh data array to a PIL Image
    
    # Example: Load multiple images (let's say 3 images)
    # image_paths = ["image1.jpg", "image2.jpg", "image3.jpg"]  # List of image file paths

    # Define the transformations
    transform = transforms.Compose([
        transforms.Resize(256),           # Resize to 256x256 (larger than 224x224 to allow for cropping)
        transforms.CenterCrop(224),       # Crop to 224x224
        transforms.ToTensor(),            # Convert to tensor (will scale pixel values to [0, 1])
        transforms.Normalize(             # Normalize with ImageNet mean and std
            mean=[0.485, 0.456, 0.406], 
            std=[0.229, 0.224, 0.225]
        ),
    ])
    
    # Prepare the list of images
    input_tensors = []
    for image in data:
        image = Image.fromarray(image)
        image_tensor = transform(image)  # Apply transformations
        input_tensors.append(image_tensor)
    
    # Stack the images into a single tensor (batch of images)
    input_batch = torch.stack(input_tensors)  # Shape will be [batch_size, 3, 224, 224]
    return input_batch


In [23]:
# Labels for image categories
labels = ['Beagle', 'Boxer', 'Bulldog', 'Dachshund', 'German_Shepherd', 'Golden_Retriever', 'Labrador_Retriever', 'Poodle', 'Rottweiler', 'Yorkshire_Terrier']
img_size = 32

In [24]:
dog_breed_train_data, dog_breed_train_label = loading_training_data('./DogBreedImageDataset/dataset')

In [33]:
dog_breed_train_data.shape

(967, 32, 32, 3)

In [25]:
test_data = dog_breed_train_data[0:5]

In [26]:
test_data.shape

(5, 32, 32, 3)

In [27]:
type(test_data)

numpy.ndarray

In [28]:
input_tensor = transform_input_resnet(test_data)

In [29]:
input_tensor.shape

torch.Size([5, 3, 224, 224])

In [38]:
labels = ['angry', 'happy', 'relaxed', 'sad']
# Load data for training
dog_emotion_train_data, dog_emotion_train_label = loading_training_data_subset('./DogEmotionPrediction/images/', 250) #100 images per class

In [39]:
dog_emotion_train_data.shape

(1000, 32, 32, 3)

## Define model

In [92]:
import torch
import torch.nn as nn
from torchvision import models

class MultiTaskResNet50(nn.Module):
    def __init__(self, num_classes_task1, num_classes_task2):
        super(MultiTaskResNet50, self).__init__()
        
        # Load the pre-trained ResNet-50 model
        self.resnet = models.resnet50(pretrained=True)
        
        # Remove the final fully connected layer (classifier) for both tasks
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-1])  # Exclude final FC layer
        
        # Task 1: A new fully connected layer for the first task
        self.fc_task1 = nn.Linear(2048, num_classes_task1)
        
        # Task 2: A new fully connected layer for the second task
        self.fc_task2 = nn.Linear(2048, num_classes_task2)
        
    def forward(self, x):
        # Pass through the ResNet backbone (everything before the final fully connected layer)
        features = self.resnet(x)  # Output size: [batch_size, 2048, 1, 1]
        
        # Flatten the output to [batch_size, 2048]
        features = features.view(features.size(0), -1)
        
        # Task 1: Pass through the task-specific classifier
        task1_output = self.fc_task1(features)
        
        # Task 2: Pass through the task-specific classifier
        task2_output = self.fc_task2(features)
        
        # Get predicted class by taking argmax (index of the maximum logit)
        task1_pred = torch.argmax(task1_output, dim=1)  # Predicted class for task 1
        task2_pred = torch.argmax(task2_output, dim=1)  # Predicted class for task 2
        
        return task1_pred, task2_pred

# Example usage:
num_classes_task1 = 10  # Number of classes for task 1
num_classes_task2 = 4   # Number of classes for task 2

# Instantiate the model
model = MultiTaskResNet50(num_classes_task1, num_classes_task2)

# Example input tensor with shape [batch_size, 3, 224, 224]
input_tensor = torch.randn(2, 3, 224, 224)  # Batch size of 2

# Forward pass
task1_output, task2_output = model(input_batch)

print("Task 1 Output:", task1_output.shape)  # Should be [batch_size, num_classes_task1]
print("Task 2 Output:", task2_output.shape)  # Should be [batch_size, num_classes_task2]


Task 1 Output: torch.Size([5])
Task 2 Output: torch.Size([5])


In [93]:
task1_output

tensor([5, 9, 9, 8, 9])

In [94]:
task2_output

tensor([0, 0, 1, 1, 1])